# 获取数据

In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from features import ensembleFeature
import itertools
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold
import pymrmr

In [3]:
##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [4]:
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
test_seq = fa_seq("data/Mouse/mouse_indep.fasta")
trainPos_seq = train_seq[0:int(len(train_seq)/2)]
trainNeg_seq = train_seq[int(len(train_seq)/2):]
testPos_seq = test_seq[0:int(len(test_seq)/2)]
testNeg_seq = test_seq[int(len(test_seq)/2):]
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
testLabel = np.append(np.ones(int(len(test_seq)/2)),np.zeros(int(len(test_seq)-len(test_seq)/2)),axis=0)

In [5]:
# 值
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
testData_PSNP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
testData_PSDP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
testData_PSTP = np.vstack((testPos,testNeg))
trainData_PSKP = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
testData_PSKP = np.hstack((testData_PSNP,testData_PSDP,testData_PSTP))



trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))
testData_PSCP_0 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))
testData_PSCP_1 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))
testData_PSCP_2 = np.vstack((testPos,testNeg))

trainData_PSCP = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))
testData_PSCP = np.hstack((testData_PSCP_0,testData_PSCP_1,testData_PSCP_2))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData_BPB = np.vstack((trainPos,trainNeg))
testData_BPB = np.vstack((testPos,testNeg))

trainData_KNN = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
testData_KNN = np.array(pd.read_csv("features/KNN_value_test.csv").iloc[:,1:])

trainData = np.hstack((trainData_PSKP,trainData_PSCP,trainData_BPB,trainData_KNN))
testData = np.hstack((testData_PSKP,testData_PSCP,testData_BPB,testData_KNN))

In [6]:
trainData_loc = trainData
from sklearn.decomposition import FastICA
ica = FastICA(n_components= 100).fit(trainData_loc)
trainData_ICA_loc = ica.transform(trainData_loc)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [7]:
trainData_ICA_loc.shape

(9126, 100)

In [8]:
list_fea = list(pd.read_csv("Result/Feature_Selection/Location_ICA_boruta_shap.csv").iloc[0:100,1])
feature = []
for i in range(len(list_fea)):
    feature.append(trainData_ICA_loc[:,int(list_fea[i][4:])])
trainData = np.array(feature).T
print(trainData.shape)


(9126, 100)


In [9]:
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,0)
for i,[train_index, test_index] in enumerate(kf.split(trainData_ICA_loc)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData_ICA_loc[train_index]
    X_test = trainData_ICA_loc[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
All = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(All[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8343436072563277

# XGBoost优化

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import plot_importance
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier
#from pymc3 import Model, Gaussian, U, MvNormal, trace_to_samples

In [59]:
trainData = trainData.reshape(9126,-1)
# 定义超参数空间
xgb_params = {
      'learning_rate': (0, 1),  # 学习率
    'n_estimators':(50,1000),
      'max_depth': (3, 15),  # 最大深度
      'min_child_weight': (1, 6),  # 最小子节点权重
      'subsample': (0.5, 1),  # 行采样比例
      'colsample_bytree': (0.2, 1),  # 列采样比例
      'gamma': (0, 5)  # 惩罚系数
  }

# 定义优化目标函数
def cv_score(**params):
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])

    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        clf_xgb = XGBClassifier(eval_metric=['logloss','auc','error'],max_depth = params['max_depth'],subsample=params['subsample'],
              n_estimators= params['n_estimators'],gamma=params['gamma'],learning_rate= params['learning_rate'],colsample_bytree=params['colsample_bytree'],min_child_weight=params['min_child_weight'])
        
        clf_xgb.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf_xgb.predict_proba(X_test)[:,1]))
    print(params['max_depth'],params['subsample'],params['n_estimators'],
          params['gamma'],params['learning_rate'],params['colsample_bytree'],params['min_child_weight'])
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 进行优化
xgb_bo = BayesianOptimization(cv_score, xgb_params, verbose=1, random_state=42)
xgb_bo.maximize()

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


10 0.5290418060840998 198 4.75357153204958 0.7319939418114051 0.49963209507789 1.7800932022121825
0.7268956546164759


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


3 0.6061695553391381 840 3.005575058716044 0.7080725777960455 0.8929409166199482 5.8495492608099715
0.7612828007208676
| 2         | 0.7613    | 0.8929    | 3.006     | 0.7081    | 3.247     | 5.85      | 840.8     | 0.6062    |


...

6 0.907387568467473 954 3.2380636374799847 0.2226111827197882 0.3773995092830664 2.48822870678956
0.8081699946041837


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


9 0.7019800500651558 956 3.0976108957341957 0.03417809533325877 0.7634410752136505 2.089150891703206
0.8264571771452351
| 29        | 0.8265    | 0.7634    | 3.098     | 0.03418   | 9.307     | 2.089     | 956.0     | 0.702     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


9 0.7800087913402379 956 2.287896548848385 0.13068469930351667 0.2667453107028225 3.496207736293896
0.8121648055537957


In [60]:
# 输出优化结果
best_params = xgb_bo.max['params']
print(best_params)
#XGBoost优化结果
#target:0.8265
#{'colsample_bytree': 0.7634410752136505, 'gamma': 3.0976108957341957, 'learning_rate': 0.03417809533325877, 
# 'max_depth': 9.30689982009134, 'min_child_weight': 2.089150891703206, 'n_estimators': 956.038699105076, 'subsample': 0.7019800500651558}

{'colsample_bytree': 0.7634410752136505, 'gamma': 3.0976108957341957, 'learning_rate': 0.03417809533325877, 'max_depth': 9.30689982009134, 'min_child_weight': 2.089150891703206, 'n_estimators': 956.038699105076, 'subsample': 0.7019800500651558}


# SVM优化

In [56]:
from thundersvm import SVC

# 定义目标函数，即SVM分类器的交叉验证准确率
def svm_evaluate(param_c, param_gamma):
#     clf = SVC(kernel='rbf', C=C, gamma=gamma)
#     scores = cross_val_score(clf, X, y, cv=5)
    
    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        #clf = SVC(kernel='polynomial', C=C, gamma=gamma)
        clf = SVC(C=param_c,gamma=param_gamma)
        
        clf.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf.decision_function(X_test)))
    print("C; ",param_c)
    print("gamma: ",param_gamma)
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 定义搜索空间
pbounds = {'param_c': (1e-3, 100), 'param_gamma': (1e-4, 20)}

In [57]:
# 实例化贝叶斯优化对象，并运行优化过程
optimizer = BayesianOptimization(f=svm_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(n_iter=25)#,allow_duplicate_points=True)

|   iter    |  target   |  param_c  | param_... |
-------------------------------------------------


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


C;  37.454637344617396
gamma:  19.014291056767682
0.790634359689379
| 1         | 0.7906    | 37.45     | 19.01     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


C;  73.1996621871987
gamma:  11.973209818092313
0.7953850947638776
| 2         | 0.7954    | 73.2      | 11.97     |


...

C;  14.531517072921568
gamma:  6.049100152360729
0.8368497309863653
| 28        | 0.8368    | 14.53     | 6.049     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


C;  11.266098258519854
gamma:  8.144142673077573
0.8368197932898612
| 29        | 0.8368    | 11.27     | 8.144     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


C;  21.5237429100287
gamma:  5.536621106133444
0.8369111400555814
| 30        | 0.8369    | 21.52     | 5.537     |


In [58]:
# 打印最优结果
print(optimizer.max)
#SVM 优化结果
#{'target': 0.8369111400555814, 'params': {'param_c': 21.5237429100287, 'param_gamma': 5.536621106133444}}

{'target': 0.8369111400555814, 'params': {'param_c': 21.5237429100287, 'param_gamma': 5.536621106133444}}


# CatBoost优化

In [11]:
import catboost as cb
from catboost import CatBoostClassifier

In [12]:
# 定义目标函数，即SVM分类器的交叉验证准确率
def catboost_evaluate(param_lr, param_depth, param_iterations,param_l2_leaf_reg):
    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        #clf = SVC(kernel='polynomial', C=C, gamma=gamma)
        clf = CatBoostClassifier(learning_rate=param_lr,depth=int(param_depth),
                                 iterations=int(param_iterations),l2_leaf_reg=param_l2_leaf_reg,task_type="GPU",verbose=0)
        
        clf.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf.predict_proba(X_test)[:,1]))
    print("learning_rate: ",param_lr)
    print("depth: ",int(param_depth))
    print("iterations: ",int(param_iterations))
    print("l2_leaf_reg: ",param_l2_leaf_reg)
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 定义搜索空间
pbounds = {'param_lr': (0.01, 0.3),'param_depth':(4,10),'param_iterations': (200,1200),
           'param_l2_leaf_reg':(1,8)}

In [13]:
# 实例化贝叶斯优化对象，并运行优化过程
optimizer = BayesianOptimization(f=catboost_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(n_iter=25)

|   iter    |  target   | param_... | param_... | param_... | param_lr  |
-------------------------------------------------------------------------


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


learning_rate:  0.18361096041714062
depth:  6
iterations:  1150
l2_leaf_reg:  6.123957592679836
0.8058093628386299
| 1         | 0.8058    | 6.247     | 1.151e+03 | 6.124     | 0.1836    |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


learning_rate:  0.2611910822747312
depth:  4
iterations:  355
l2_leaf_reg:  1.4065852851773961
0.8080474686020738
| 2         | 0.808     | 4.936     | 356.0     | 1.407     | 0.2612    |


...

learning_rate:  0.01
depth:  7
iterations:  722
l2_leaf_reg:  4.653585581898909
0.8127685874269035
| 28        | 0.8128    | 7.308     | 722.7     | 4.654     | 0.01      |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


learning_rate:  0.1342097963348748
depth:  6
iterations:  723
l2_leaf_reg:  5.740931297059213
0.8147340234199743
| 29        | 0.8147    | 6.415     | 723.4     | 5.741     | 0.1342    |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


learning_rate:  0.251374744901722
depth:  7
iterations:  722
l2_leaf_reg:  5.956838640290016
0.8011260599445569
| 30        | 0.8011    | 7.625     | 722.4     | 5.957     | 0.2514    |


In [14]:
# 打印最优结果
print(optimizer.max)
#CatBoost 优化结果
#{'target': 0.820760529873312, 'params': {'param_depth': 6.7793582738598595, 'param_iterations': 723.4748699393687, 'param_l2_leaf_reg': 4.967918190058223, 'param_lr': 0.049933322436247485}}

{'target': 0.820760529873312, 'params': {'param_depth': 6.7793582738598595, 'param_iterations': 723.4748699393687, 'param_l2_leaf_reg': 4.967918190058223, 'param_lr': 0.049933322436247485}}


# LGBM优化

In [16]:
from lightgbm import LGBMClassifier

In [17]:
# 定义目标函数，即SVM分类器的交叉验证准确率
def lgbm_evaluate(param_lr,param_depth,param_n_estimators,param_min_split_gain,
                  param_subsample,param_colsample_bytree,param_lambda_l1,param_num_leaves):
    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        #clf = SVC(kernel='polynomial', C=C, gamma=gamma)
        clf = LGBMClassifier(learning_rate=param_lr,max_depth=int(param_depth),n_estimators=int(param_n_estimators),
                             min_split_gain=param_min_split_gain,subsample=param_subsample,
                             colsample_bytree=param_colsample_bytree,lambda_l1=param_lambda_l1,
                             num_leaves=int(param_num_leaves),verbose=0)
        
        clf.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf.predict_proba(X_test)[:,1]))
        
    print("learning_rate: ",param_lr)
    print("max_depth: ",int(param_depth))
    print("n_estimators: ",int(param_n_estimators))
    print("min_split_gain: ",param_min_split_gain)
    print("subsample: ",param_subsample)
    print("colsample_bytree: ",param_colsample_bytree)
    print("lambda_l1: ",param_lambda_l1)
    print("num_leaves: ",int(param_num_leaves))
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 定义搜索空间
pbounds = {'param_lr': (0.01, 0.2),'param_depth':(3,6),'param_n_estimators': (200,1200),
           'param_min_split_gain':(0.01,0.2),'param_subsample':(0.7,1),'param_colsample_bytree':(0.8,1),
          'param_lambda_l1':(0,10),'param_num_leaves':(0,63)}

In [59]:
# 实例化贝叶斯优化对象，并运行优化过程
# optimizer = BayesianOptimization(f=lgbm_evaluate, pbounds=pbounds, random_state=42)
# optimizer.maximize(n_iter=50)

In [19]:
# 打印最优结果
print(optimizer.max)
#LightGBM 优化结果
#{'target': 0.8226275784103461, 'params': {'param_colsample_bytree': 0.8608484485919076, 'param_depth': 4.574269294896713, 'param_lambda_l1': 4.319450186421157, 'param_lr': 0.06533353663762796, 'param_min_split_gain': 0.1262520499972521, 'param_n_estimators': 339.4938606520418, 'param_num_leaves': 18.405112857718745, 'param_subsample': 0.8099085529881075}}

{'target': 0.8226275784103461, 'params': {'param_colsample_bytree': 0.8608484485919076, 'param_depth': 4.574269294896713, 'param_lambda_l1': 4.319450186421157, 'param_lr': 0.06533353663762796, 'param_min_split_gain': 0.1262520499972521, 'param_n_estimators': 339.4938606520418, 'param_num_leaves': 18.405112857718745, 'param_subsample': 0.8099085529881075}}


# GBDT优化

In [47]:
from sklearn.ensemble import GradientBoostingClassifier

In [48]:
# 定义目标函数，即SVM分类器的交叉验证准确率
def gbdt_evaluate(param_n_estimators,param_learning_rate,param_max_depth,param_min_samples_split):
    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        #clf = SVC(kernel='polynomial', C=C, gamma=gamma)
        clf = GradientBoostingClassifier(n_estimators=int(param_n_estimators),learning_rate=param_learning_rate,
                                 max_depth=param_max_depth,min_samples_split=int(param_min_samples_split))
        
        clf.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf.predict_proba(X_test)[:,1]))
        
    print("param_n_estimators: ",param_n_estimators)
    print("param_learning_rate: ",param_learning_rate)
    print("param_max_depth: ",param_max_depth)
    print("param_min_samples_split: ",param_min_samples_split)
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 定义搜索空间
pbounds = {'param_n_estimators': (50, 1200),'param_learning_rate':(0.01,0.2),'param_max_depth': (4,30),'param_min_samples_split':(10,100)}

In [ ]:
# 实例化贝叶斯优化对象，并运行优化过程
optimizer = BayesianOptimization(f=gbdt_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(n_iter=25)

|   iter    |  target   | param_... | param_... | param_... | param_... |
-------------------------------------------------------------------------


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  738.4572568265921
param_learning_rate:  0.08116262258099886
param_max_depth:  28.71857196665782
param_min_samples_split:  75.87945476302646
0.8160864556224012
| 1         | 0.8161    | 0.08116   | 28.72     | 75.88     | 738.5     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  1046.1025676411755
param_learning_rate:  0.039643541684062936
param_max_depth:  8.055857528741269
param_min_samples_split:  15.227525095137953
0.8207917095594599
| 2         | 0.8208    | 0.03964   | 8.056     | 15.23     | 1.046e+03 |


...

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  262.3273259494746
param_learning_rate:  0.14130060442045975
param_max_depth:  11.06246271313361
param_min_samples_split:  27.15488497670048
0.8134746711465398
| 6         | 0.8135    | 0.1413    | 11.06     | 27.15     | 262.3     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  245.21608303105396
param_learning_rate:  0.2
param_max_depth:  4.0
param_min_samples_split:  16.769209750544523
0.81175460773675
| 7         | 0.8118    | 0.2       | 4.0       | 16.77     | 245.2     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [58]:
# 打印最优结果
print(optimizer.max)
#GBDT 优化结果
#{'target': 0.8231106871675493, 'params': {'param_learning_rate': 0.17659200841509837, 'param_max_depth': 8.972115135946286, 'param_min_samples_split': 23.339078467861945, 'param_n_estimators': 1061.9984805178046}}

{'target': 0.8231106871675493, 'params': {'param_learning_rate': 0.17659200841509837, 'param_max_depth': 8.972115135946286, 'param_min_samples_split': 23.339078467861945, 'param_n_estimators': 1061.9984805178046}}


# AdaBoost优化

In [20]:
from sklearn.ensemble import AdaBoostClassifier

In [40]:
# 定义目标函数，即SVM分类器的交叉验证准确率
def ada_evaluate(param_n_estimators,param_learning_rate):#,param_depth,param_min_samples_split,param_min_samples_leaf,param_min_weight_fraction_leaf):
    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        #clf = SVC(kernel='polynomial', C=C, gamma=gamma)
        clf = AdaBoostClassifier(n_estimators=int(param_n_estimators),learning_rate=param_learning_rate)#,depth=param_depth,
                                #min_samples_split=param_min_samples_split,
                                 #min_samples_leaf=param_min_samples_leaf,
                                #min_weight_fraction_leaf=param_min_weight_fraction_leaf)
        clf.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf.predict_proba(X_test)[:,1]))
    
    print("param_n_estimators: ",param_n_estimators)
    print("param_learning_rate: ",param_learning_rate)
#     print("param_depth: ",param_depth)
#     print("param_min_samples_split: ",param_min_samples_split)
#     print("param_min_samples_leaf: ",param_min_samples_leaf)
#     print("param_min_weight_fraction_leaf: ",param_min_weight_fraction_leaf)
    
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 定义搜索空间
pbounds = {'param_n_estimators':(200,800),'param_learning_rate':(0.01,0.2)}#, 'param_depth': (10, 100),'param_min_samples_split':(1,10),'param_min_samples_leaf': (0.01,2),
           #'param_min_weight_fraction_leaf':(0,1)}

In [41]:
# 实例化贝叶斯优化对象，并运行优化过程
optimizer = BayesianOptimization(f=ada_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(n_iter=25)

|   iter    |  target   | param_... | param_... |
-------------------------------------------------


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  770.4285838459497
param_learning_rate:  0.08116262258099886
0.8132235609585662
| 1         | 0.8132    | 0.08116   | 770.4     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  559.195090518222
param_learning_rate:  0.14907884894416698
0.8163124487117068
| 2         | 0.8163    | 0.1491    | 559.2     |


...

param_n_estimators:  525.7997010897484
param_learning_rate:  0.18721744424252812
0.8167334875218104
| 28        | 0.8167    | 0.1872    | 525.8     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  633.6385080040905
param_learning_rate:  0.18733244969706583
0.8169330055698586
| 29        | 0.8169    | 0.1873    | 633.6     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


param_n_estimators:  433.4631751344057
param_learning_rate:  0.2
0.8172021666078146
| 30        | 0.8172    | 0.2       | 433.5     |


In [42]:
# 打印最优结果
print(optimizer.max)
#Adaboost 优化结果
#{'target': 0.817646776977616, 'params': {'param_learning_rate': 0.1608548813008134, 'param_n_estimators': 781.5935263096853}}

{'target': 0.817646776977616, 'params': {'param_learning_rate': 0.1608548813008134, 'param_n_estimators': 781.5935263096853}}


# ExtraTree优化

In [43]:
from sklearn.ensemble import ExtraTreesClassifier

In [44]:
# 定义目标函数，即SVM分类器的交叉验证准确率
def extra_evaluate(param_n_estimators,param_min_samples_split,param_max_depth):
    Auc = []
    kf = KFold(10,True,0)
    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
        X_train = trainData[train_index]
        X_test = trainData[test_index]
        Y_train = trainLabel[train_index]
        Y_test = trainLabel[test_index]
        if param_min_samples_split>1:
            param_min_samples_split=int(param_min_samples_split)
        clf = ExtraTreesClassifier(n_estimators=int(param_n_estimators) ,
                                   min_samples_split=param_min_samples_split,max_depth=param_max_depth)
        
        clf.fit(X_train,Y_train)
        Auc.append(roc_auc_score(Y_test,clf.predict_proba(X_test)[:,1]))
        
    print("n_estimators: ",param_n_estimators)
    print("min_samples_split: ",param_min_samples_split)
    print("max_depth: ",param_max_depth)
    print(sum(Auc) / len(Auc))
    return sum(Auc) / len(Auc)

# 定义搜索空间
pbounds = {'param_n_estimators': (10, 1200),'param_min_samples_split':(2,20),'param_max_depth': (10,50)}

In [45]:
# 实例化贝叶斯优化对象，并运行优化过程
optimizer = BayesianOptimization(f=extra_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(n_iter=25)

|   iter    |  target   | param_... | param_... | param_... |
-------------------------------------------------------------


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


n_estimators:  881.0727907555721
min_samples_split:  19
max_depth:  24.9816047538945
0.8153310966819637
| 1         | 0.8153    | 24.98     | 19.11     | 881.1     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


n_estimators:  195.63347920008115
min_samples_split:  4
max_depth:  33.946339367881464
0.8092615249464051
| 2         | 0.8093    | 33.95     | 4.808     | 195.6     |


...

n_estimators:  701.5359117391474
min_samples_split:  19
max_depth:  48.797517785015415
0.8149966410771212
| 28        | 0.815     | 48.8      | 19.54     | 701.5     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


n_estimators:  611.0193695952843
min_samples_split:  20
max_depth:  50.0
0.8156689548040333
| 29        | 0.8157    | 50.0      | 20.0      | 611.0     |


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


n_estimators:  1198.9301764395077
min_samples_split:  3
max_depth:  49.52796126612924
0.8173092504363721
| 30        | 0.8173    | 49.53     | 3.785     | 1.199e+03 |


In [46]:
# 打印最优结果
print(optimizer.max)
#ExtraTree 优化结果
#{'target': 0.8179219859927791, 'params': {'param_max_depth': 40.866576594703616, 'param_min_samples_split': 4.295678174414615, 'param_n_estimators': 1164.0984611812682}}

{'target': 0.8179219859927791, 'params': {'param_max_depth': 40.866576594703616, 'param_min_samples_split': 4.295678174414615, 'param_n_estimators': 1164.0984611812682}}
